In [5]:
import cohere
co = cohere.Client('lfE7VQJQhLyND4U5JxlMVJin2H3xBCGGz3k8cvSA')

In [ ]:
import pandas as pd
from collections import defaultdict

In [ ]:
dataset = pd.read_csv('muse_v3.csv')[['artist', 'track', 'seeds', 'number_of_emotion_tags']]
dataset.head()

,artist,track,seeds,number_of_emotion_tags
0,Eminem,'Till I Collapse,['aggressive'],6
1,Metallica,St. Anger,['aggressive'],8
2,Rick Ross,Speedin',['aggressive'],1
3,M.I.A.,Bamboo Banga,"['aggressive', 'fun', 'sexy', 'energetic']",13
4,Dope,Die MF Die,['aggressive'],7


Getting lyrics

In [ ]:
from lyricsgenius import Genius

token = open('TOKEN').readline()
genius = Genius(token)

In [ ]:
from collections import defaultdict
import re

AMT_OF_SONGS = 2000

i, songs = 0, {}
emotions = set()
musics_by_emotion = defaultdict(list)
for name, artist, seeds, song in zip(dataset['track'], dataset['artist'], dataset['seeds'], dataset['track']):
    artist_name = '-'.join(re.sub(r'[^a-zA-Z0-9_ ]+', '', artist).split(' '))
    song_name = '-'.join(re.sub(r'[^a-zA-Z0-9_ ]+', '', song).split(' '))

    url = f"https://genius.com/{artist_name}-{song_name}-lyrics"
    try:
        lyrics = genius.lyrics(song_url=url)
        lyrics = '\n'.join(lyrics.split('\n')[1:]) # removing contributors line
        songs[(artist, song)] = lyrics

        for emotion in seeds[1:-1].split(','):
            x = emotion.strip()[1:-1]
            emotions.add(x)
            musics_by_emotion[x].append((artist, name))
        i += 1
        if i == AMT_OF_SONGS: 
            break
    except: 
        continue

Assigning emotions

In [ ]:
top_emotions = []
for emotion, tracks in musics_by_emotion.items():
    top_emotions.append([emotion, len(tracks)])

In [ ]:
top_emotions = sorted(top_emotions, key=lambda x: -x[1])

In [ ]:
from cohere.responses.classify import Example
from random import sample

examples = [] 
for emotion, amt in top_emotions:
    if amt < 5: break
    songs_sample = sample(musics_by_emotion[emotion], 5)
    # print(sample)
    for song in songs_sample:
        lrx = songs[song] 
        while not lrx:
            lrx = songs[sample(musics_by_emotion[emotion], 1)[0]]     
        examples.append(Example(lrx, emotion))


In [ ]:
lyrics = genius.lyrics(song_url='https://genius.com/slayer-raining-blood-lyrics')
lyrics = '\n'.join(lyrics.split('\n')[1:])
inputs = [lyrics]

In [ ]:
response = co.classify(
  model='large',
  inputs=inputs,
  examples=examples,
)

In [ ]:
print(response.classifications)

[Classification<prediction: "negative", confidence: 0.15187037, labels: {'aggressive': LabelPrediction(confidence=0.0071394774), 'angry': LabelPrediction(confidence=0.0020761038), 'angst-ridden': LabelPrediction(confidence=8.489971e-05), 'apocalyptic': LabelPrediction(confidence=0.00504946), 'bitter': LabelPrediction(confidence=0.001380384), 'bittersweet': LabelPrediction(confidence=9.683359e-05), 'brash': LabelPrediction(confidence=0.0002146803), 'brooding': LabelPrediction(confidence=0.005594352), 'cathartic': LabelPrediction(confidence=0.0018577385), 'cold': LabelPrediction(confidence=0.0013831706), 'comic': LabelPrediction(confidence=0.00019707909), 'confrontational': LabelPrediction(confidence=4.6491627e-05), 'cynical': LabelPrediction(confidence=0.0018596295), 'dark': LabelPrediction(confidence=0.056621358), 'defiant': LabelPrediction(confidence=0.00054946134), 'demonic': LabelPrediction(confidence=0.027741263), 'desperate': LabelPrediction(confidence=0.10413744), 'dramatic': Lab

In [ ]:
prompt = f"rewrite the following lyrics, but with a positive mood:\n{lyrics}"
response = co.generate(  
    model='command-nightly',  
    prompt = prompt,
    temperature=0.750)

intro_paragraph = response.generations[0].text

In [ ]:
for line in intro_paragraph.split('\n'):
    print(line)

 [Verse]
I'm free from the chains of my past,
Strengthened by the struggles I faced.
No longer at the mercy of others,
I've risen like a fiery phoenix.

The sky above isn't a warning,
It's a canvas of infinite possibilities.
I'm reaching for the stars,
Ready to soar to new heights.

[Bridge]
I am the architect of my destiny,
Building a future with my own two hands.
I pierce through negativity,
Leaving my past traumas behind.
I'm guided by love and light,
Unshakable, I stand strong.

[Chorus]
Reigning in a world of vibrant colors,
I'm bathed in the rain of prosperity.
It's a world where I can be myself,
Fulfilling my destiny with a joyful spirit.
In this new realm of infinite beauty,
I stand tall, with head held high.

This rewrite focuses on the theme of personal growth, resilience, and the power to overcome challenges. It highlights the protagonist's transformation from a state of being trapped and powerless to a place of strength, self-actualization, and positive outlook. 
